In [154]:
import pickle
import urllib.request as req
import os
import pandas as pd
import numpy as np
import os

In [59]:
def download_images(dataframe,out_folder):
    ids = dataframe.index.values
    for image_id in ids:
        url = dataframe.loc[image_id]['image_url']
        if os.path.isfile(out_folder + '/' + str(image_id) + '.jpg') == False:
            r = req.urlretrieve(url, out_folder + '/' + str(image_id) + '.jpg')
    return True

In [60]:
def return_image_name_col(df):
    ids = df.index.values
    names = []
    for i in ids:
        names.append(str(i)+'.jpg')
    return np.array(names)

In [ ]:
def return_ids_and_valid_files(df):
    ids = df.index.values
    valid_ids = []
    names = []
    

In [61]:
#load training set
pickle_in_train = open("all_books_train.pickle","rb")
train = pickle.load(pickle_in_train)

In [62]:
#load test set
pickle_in_test = open("all_books_test.pickle","rb")
test = pickle.load(pickle_in_test)

In [63]:
train.shape

(36390, 14)

In [64]:
test.shape

(12131, 14)

In [65]:
#show training data
train.head()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url,genres_list,genres_cut
6377,Nicole Jacquelyn,"""How is it, that someone can make decision aft...",NaN,Kindle Edition,NaN,305 pages,4.02,17962,1159,Craving Constellations,Romance|Romance|Contemporary Romance|Contemporary,https://images.gr-assets.com/books/1377563339l...,"[Romance, Contemporary, Contemporary Romance]","[Romance, Contemporary, Contemporary Romance]"
19880,Adam Schell,A village in Tuscany is the setting for this j...,NaN,Hardcover,9.78039E+12,340 pages,3.74,646,161,"Tomato Rhapsody: A Fable of Love, Lust & Forbi...",Fiction|Historical|Historical Fiction|Cultural...,https://images.gr-assets.com/books/1320422913l...,"[Historical Fiction, Fiction, Food and Drink, ...","[Historical Fiction, Fiction, Romance, Histori..."
23352,Robert Moss,Have you ever said something was “only a dream...,NaN,Hardcover,9.78158E+12,224 pages,4.04,345,41,"The Three ""Only"" Things: Tapping the Power of ...",Nonfiction|Spirituality|Psychology|Spiritualit...,https://images.gr-assets.com/books/1328746102l...,"[Nonfiction, New Age, Psychology, Spirituality]",[Nonfiction]
16021,David Ambrose,A train of seemingly random events - coinciden...,NaN,Unknown Binding,9.78074E+12,NaN,3.35,166,18,Coincidence,Fiction|Suspense|Historical|Science Fiction|My...,https://images.gr-assets.com/books/1408924887l...,"[Suspense, Fiction, Mystery, Historical, Scien...","[Fiction, Mystery, Historical, Science Fiction]"
49981,Richard North Patterson,When the body of eleven-year-old Thuy Sen is f...,NaN,Hardcover,9.78035E+12,463 pages,3.89,2400,154,Conviction,Fiction|Mystery|Thriller|Legal Thriller|Thriller,https://images.gr-assets.com/books/1399495278l...,"[Thriller, Legal Thriller, Fiction, Mystery]","[Thriller, Fiction, Mystery]"


In [68]:
#download training images
download_images(train,"train_images")

True

In [69]:
#download test images
download_images(test, "test_images")

True

In [25]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

Using TensorFlow backend.


In [89]:
import itertools

In [78]:
import warnings
warnings.filterwarnings('ignore')

In [79]:
train_filename_col = return_image_name_col(train)

In [80]:
train_df = train[['genres_cut']]
train_df['file_name'] = train_filename_col

In [82]:
train_df.head()

,genres_cut,file_name
6377,"[Romance, Contemporary, Contemporary Romance]",6377.jpg
19880,"[Historical Fiction, Fiction, Romance, Histori...",19880.jpg
23352,[Nonfiction],23352.jpg
16021,"[Fiction, Mystery, Historical, Science Fiction]",16021.jpg
49981,"[Thriller, Fiction, Mystery]",49981.jpg


In [83]:
test_filename_col = return_image_name_col(test)
test_df = test[['genres_cut']]
test_df['file_name'] = test_filename_col
test_df.head()

,genres_cut,file_name
9102,"[Romance, Fiction]",9102.jpg
34116,"[History, Biography, Nonfiction]",34116.jpg
53879,"[Crime, Fiction, Mystery, Thriller]",53879.jpg
30327,"[Childrens, Fantasy]",30327.jpg
36487,"[Young Adult, Fantasy]",36487.jpg


In [248]:
train_df_valid = train_df.copy()

In [249]:
for index,row in train_df.iterrows():
    path = 'train_images/' + row['file_name']
    if os.stat(path).st_size < 1024:
        train_df_valid.drop(index = index,inplace=True)
    

In [250]:
train_df_valid.shape

(36299, 2)

In [253]:
test_df_valid = test_df.copy()
for index,row in test_df.iterrows():
    path = 'test_images/' + row['file_name']
    if os.stat(path).st_size < 1024:
        test_df_valid.drop(index = index,inplace=True)

In [254]:
test_df_valid.shape

(12096, 2)

In [255]:
all_classes = np.unique(list(itertools.chain.from_iterable(train_df['genres_cut'])))

In [292]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)
test_datagen=ImageDataGenerator(rescale=1./255.)

In [293]:
train_generator=datagen.flow_from_dataframe(
dataframe=train_df_valid,
directory="train_images",
x_col="file_name",
y_col="genres_cut",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
classes=list(all_classes),
target_size=(100,100))

Found 36299 validated image filenames belonging to 25 classes.


In [294]:
valid_generator=datagen.flow_from_dataframe(
dataframe=train_df_valid,
directory="train_images",
x_col="file_name",
y_col="genres_cut",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
classes=list(all_classes),
target_size=(100,100))

Found 36299 validated image filenames belonging to 25 classes.


In [295]:
test_generator=test_datagen.flow_from_dataframe(
dataframe=test_df_valid,
directory="test_images",
x_col="file_name",
y_col=None,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(100,100))

Found 12096 validated image filenames.


In [296]:
#bulid the model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(100,100,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(25, activation='sigmoid'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])

In [297]:
#fitting the model
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [298]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Epoch 1/10
1134/1134 [==============================] - 835s 737ms/step - loss: 0.3374 - acc: 0.8788 - val_loss: 0.3402 - val_acc: 0.8827
Epoch 2/10
1134/1134 [==============================] - 903s 796ms/step - loss: 0.3222 - acc: 0.8822 - val_loss: 0.3210 - val_acc: 0.8843
Epoch 3/10
1134/1134 [==============================] - 858s 756ms/step - loss: 0.3148 - acc: 0.8839 - val_loss: 0.3081 - val_acc: 0.8858
Epoch 4/10
1134/1134 [==============================] - 816s 720ms/step - loss: 0.3091 - acc: 0.8850 - val_loss: 0.3034 - val_acc: 0.8861
Epoch 5/10
1134/1134 [==============================] - 901s 795ms/step - loss: 0.3047 - acc: 0.8861 - val_loss: 0.2967 - val_acc: 0.8883
Epoch 6/10
1134/1134 [==============================] - 940s 829ms/step - loss: 0.3006 - acc: 0.8867 - val_loss: 0.2927 - val_acc: 0.8889
Epoch 7/10
1134/1134 [==============================] - 769s 678ms/step - loss: 0.2965 - acc: 0.8878 - val_loss: 0.2881 - val_acc: 0.8897
Epoch 8/10
1134/1134 [============

In [299]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

12096/12096 [==============================] - 161s 13ms/step


In [307]:
pred_bool = (pred > 0.3)

In [308]:
predictions=[]
labels = train_generator.class_indices
labels = dict((v,k) for k,v in labels.items())
for row in pred_bool:
    l=[]
    for index,cls in enumerate(row):
        if cls:
            l.append(labels[index])
    predictions.append(",".join(l))
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results_keras.csv",index=False)